# SageMaker 跟踪

>[Amazon SageMaker](https://aws.amazon.com/sagemaker/) 是一项全托管服务，用于快速轻松地构建、训练和部署机器学习 (ML) 模型。

>[Amazon SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) 是 `Amazon SageMaker` 的一项功能，允许您组织、跟踪、比较和评估 ML 实验和模型版本。

本笔记本展示了如何使用 LangChain 回调将提示和其他 LLM 超参数记录并跟踪到 `SageMaker Experiments` 中。在这里，我们使用不同的场景来展示该功能：

* **场景 1**：*单一 LLM* - 使用单个 LLM 模型根据给定提示生成输出的情况。
* **场景 2**：*顺序链* - 使用两个 LLM 模型的顺序链的情况。
* **场景 3**：*带工具的代理（思维链）* - 除了 LLM 之外，还使用多种工具（搜索和数学）的情况。


在本笔记本中，我们将创建一个单一实验来记录每个场景中的提示。

## 安装和设置

In [ ]:
%pip install --upgrade --quiet  sagemaker
%pip install --upgrade --quiet  langchain-openai
%pip install --upgrade --quiet  google-search-results

首先，设置所需的 API 密钥

* OpenAI：https://platform.openai.com/account/api-keys（用于 OpenAI LLM 模型）
* Google SERP API：https://serpapi.com/manage-api-key（用于 Google 搜索工具）

In [ ]:
import os

## 在下面添加您的 API 密钥
os.environ["OPENAI_API_KEY"] = "<在此添加密钥>"
os.environ["SERPAPI_API_KEY"] = "<在此添加密钥>"

In [ ]:
from langchain_community.callbacks.sagemaker_callback import SageMakerCallbackHandler

In [ ]:
from langchain.agents import initialize_agent, load_tools
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.experiments.run import Run
from sagemaker.session import Session

## LLM 提示跟踪

In [ ]:
# LLM 超参数
HPARAMS = {
    "temperature": 0.1,
    "model_name": "gpt-3.5-turbo-instruct",
}

# 用于保存提示日志的存储桶（使用 `None` 表示保存在默认存储桶中，或者更改它）
BUCKET_NAME = None

# 实验名称
EXPERIMENT_NAME = "langchain-sagemaker-tracker"

# 使用指定存储桶创建 SageMaker 会话
session = Session(default_bucket=BUCKET_NAME)

### 场景 1 - LLM

In [ ]:
RUN_NAME = "run-scenario-1"
PROMPT_TEMPLATE = "给我讲个关于{topic}的笑话"
INPUT_VARIABLES = {"topic": "鱼"}

In [ ]:
with Run(
    experiment_name=EXPERIMENT_NAME, run_name=RUN_NAME, sagemaker_session=session
) as run:
    # 创建 SageMaker 回调
    sagemaker_callback = SageMakerCallbackHandler(run)

    # 定义带回调的 LLM 模型
    llm = OpenAI(callbacks=[sagemaker_callback], **HPARAMS)

    # 创建提示模板
    prompt = PromptTemplate.from_template(template=PROMPT_TEMPLATE)

    # 创建 LLM 链
    chain = LLMChain(llm=llm, prompt=prompt, callbacks=[sagemaker_callback])

    # 运行链
    chain.run(**INPUT_VARIABLES)

    # 重置回调
    sagemaker_callback.flush_tracker()

### 场景 2 - 顺序链

In [ ]:
RUN_NAME = "run-scenario-2"

PROMPT_TEMPLATE_1 = """你是一位剧作家。给定剧本标题，你的任务是为该标题写一个剧情概要。
标题：{title}
剧作家：这是上述剧本的概要："""
PROMPT_TEMPLATE_2 = """你是纽约时报的戏剧评论家。给定剧本概要，你的任务是为该剧本写一篇评论。
剧本概要：{synopsis}
纽约时报戏剧评论家对上述剧本的评论："""

INPUT_VARIABLES = {
    "input": "关于推动游戏设计边界的优秀电子游戏的纪录片"
}

In [ ]:
with Run(
    experiment_name=EXPERIMENT_NAME, run_name=RUN_NAME, sagemaker_session=session
) as run:
    # 创建 SageMaker 回调
    sagemaker_callback = SageMakerCallbackHandler(run)

    # 为链创建提示模板
    prompt_template1 = PromptTemplate.from_template(template=PROMPT_TEMPLATE_1)
    prompt_template2 = PromptTemplate.from_template(template=PROMPT_TEMPLATE_2)

    # 定义带回调的 LLM 模型
    llm = OpenAI(callbacks=[sagemaker_callback], **HPARAMS)

    # 创建链1
    chain1 = LLMChain(llm=llm, prompt=prompt_template1, callbacks=[sagemaker_callback])

    # 创建链2
    chain2 = LLMChain(llm=llm, prompt=prompt_template2, callbacks=[sagemaker_callback])

    # 创建顺序链
    overall_chain = SimpleSequentialChain(
        chains=[chain1, chain2], callbacks=[sagemaker_callback]
    )

    # 运行整体顺序链
    overall_chain.run(**INPUT_VARIABLES)

    # 重置回调
    sagemaker_callback.flush_tracker()

### 场景 3 - 带工具的代理

In [ ]:
RUN_NAME = "run-scenario-3"
PROMPT_TEMPLATE = "世界上最长寿的人是谁？他们当前的年龄的1.51次方是多少？"

In [ ]:
with Run(
    experiment_name=EXPERIMENT_NAME, run_name=RUN_NAME, sagemaker_session=session
) as run:
    # 创建 SageMaker 回调
    sagemaker_callback = SageMakerCallbackHandler(run)

    # 定义带回调的 LLM 模型
    llm = OpenAI(callbacks=[sagemaker_callback], **HPARAMS)

    # 定义工具
    tools = load_tools(["serpapi", "llm-math"], llm=llm, callbacks=[sagemaker_callback])

    # 使用所有工具初始化代理
    agent = initialize_agent(
        tools, llm, agent="zero-shot-react-description", callbacks=[sagemaker_callback]
    )

    # 运行代理
    agent.run(input=PROMPT_TEMPLATE)

    # 重置回调
    sagemaker_callback.flush_tracker()

## 加载日志数据

一旦提示被记录下来，我们可以轻松地加载它们并将其转换为Pandas DataFrame，如下所示。

In [ ]:
# 加载
logs = ExperimentAnalytics(experiment_name=EXPERIMENT_NAME)

# 转换为pandas数据框
df = logs.dataframe(force_refresh=True)

print(df.shape)
df.head()

如上所示，实验中有三个运行（行），对应于每个场景。每次运行都会以JSON格式记录提示和相关的LLM设置/超参数，并保存在s3存储桶中。请随时从每个JSON路径加载并探索日志数据。